## 基于llamaindex去构建一个完整的流程
- 将用于检索的块与用于合成的块儿分离
- 大型文档集的结构化检索
- 根据任务动态检索块
- 优化上下文嵌入

### 向量数据库的构建

In [ ]:
from dotenv import load_dotenv, find_dotenv

# 加载.env文件获取API-key
load_dotenv(find_dotenv())

In [1]:
# 文档读取
from llama_index.core import SimpleDirectoryReader
# 在底层，这会将您的文档拆分为节点对象，这些对象类似于文档（它们包含文本和元数据）但与其父文档有关系。
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

# 读取文档
# documents = SimpleDirectoryReader("./data").load_data()


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/dummy/anaconda3/envs/pytorch/lib/python3.8/site
[nltk_data]     -packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# SimpleDirectoryReader(input_files=["./data/english.pdf"])

In [9]:
# def get_meta(file_path):
#     return {"foo": "bar", "file_path": file_path}

dic = SimpleDirectoryReader(input_files=["./data/english.pdf"])
docs = dic.load_data()
print(f'共有{len(docs)}页')


print(docs[0].text)

共有25页
2024考研英语阅读预测
李达
第一节历年阅读来源、话题整理
年份话题 杂志/期刊出处
2023text1环保-假草坪的好坏纷争； 《泰晤士报》
text2社会-美国国家公园面临的问题 《洛杉矶时报》
text3科技-网络改变人的记忆； 《美国时代周刊》
text4教育-青少年时期的矛盾行为； 《华尔街日报》
2022text1环保-环境友好型鸡蛋和可再生农场 《彭博观点》
text2社会-很多美国人选择在退休后继续工作； 《今日美国》
text3法制-如何远离“灰暗模式（侵犯用户隐私）”；《福布斯》
text4教育-伦理道德课程可否改变人们行为 《科学美国人》
2021text1社会-是时候开展技能培训了； 《环球邮报》
text2经济-英国脱欧之后实现食物自给自足不过是一场梦；《新科学家》
text3经济-科技巨头“先买再杀”策略收到严格审查《金融时报》
text4心理-直觉有时比深思更准确 《今日心理学》
2020text1科技-老鼠与机器鼠交朋友并就其脱险 《新科学家》
text2经济-为什么CEO理应获得亿万美元高新； 《时代周刊》
text3环保-“清新空气区”； 《卫报》
text4社会-Z时代的工作 《新闻周刊》
2019text1心理-内疚感可能是好事； 《大西洋月刊》


In [3]:
pip install nest_asyncio

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nest_asyncio
nest_asyncio.apply()
# bring in our LLAMA_CLOUD_API_KEY
from dotenv import load_dotenv
load_dotenv()

# bring in deps
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# set up parser
parser = LlamaParse(
    result_type="markdown"  # "markdown" and "text" are available
)

# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=['data/english.pdf'], file_extractor=file_extractor).load_data()
print(documents[0])

Started parsing the file under job_id 620ef72d-e445-400d-908d-a602368c0b51
Doc ID: d518d5c2-fdac-4e87-bea2-00b36b0b2679
Text: # 2024考研英语阅读预测  # 李达  # 第一节     历年阅读来源、话题整理  |年份|话题|杂志/期刊出处|
|---|---|---| |2023|环保-假草坪的好坏纷争|《泰晤士报》| |2023|社会-美国国家公园面临的问题|《洛杉矶时报》|
|2023|科技-网络改变人的记忆|《美国时代周刊》| |2023|教育-青少年时期的矛盾行为|《华尔街日报》| |2022|环保-
环境友好型鸡蛋和可再生农场|《彭博观点》| |2022|社会-很多美国人选择在退休后继续工作|《今日美国》| |2022|法制-
如何远离“灰暗模式（侵犯用户隐私）”|《福布斯》| |2022|教育-伦理道德课程可否改变人们行为|《科学美国人》| |2022|社会-
是时候开展技能培训了|《环球邮报》|...


In [ ]:
# 使用文件构建 
from pathlib import Path
from llama_index import download_loader

# MarkdownReader = download_loader("MarkdownReader") # 方法已被删除
# reader = MarkdownReader()

# documents = reader.load_data(Path("data/review.md"))

In [ ]:
# 对文档进行切分，将切分后的片段转化为embedding向量，构建向量索引
index = VectorStoreIndex.from_documents(documents, transformations=[SentenceSplitter(chunk_size=256)])


In [ ]:
# 将embedding向量和向量索引存储到文件中
index.storage_context.persist(persist_dir='doc_emb')


In [ ]:
# 构建查询引擎
query_engine = index.as_query_engine(similarity_top_k=5)


In [ ]:
# 查询获得答案
response = query_engine.query("新进人员报到的主要流程是什么？")

## 调优